<!-- <img src='images/besm.png' width='250px'> -->
<style>
    
@font-face {font-family: "B Lotus"; src: url("//db.onlinewebfonts.com/t/1605a655ba0a3246ce5eca3eaff6c5c2.eot"); src: url("//db.onlinewebfonts.com/t/1605a655ba0a3246ce5eca3eaff6c5c2.eot?#iefix") format("embedded-opentype"), url("//db.onlinewebfonts.com/t/1605a655ba0a3246ce5eca3eaff6c5c2.woff2") format("woff2"), url("//db.onlinewebfonts.com/t/1605a655ba0a3246ce5eca3eaff6c5c2.woff") format("woff"), url("//db.onlinewebfonts.com/t/1605a655ba0a3246ce5eca3eaff6c5c2.ttf") format("truetype"), url("//db.onlinewebfonts.com/t/1605a655ba0a3246ce5eca3eaff6c5c2.svg#B Lotus") format("svg"); }


</style>

<h1 style='direction:rtl; font-family: "B Lotus";'> 
    <center style ='font-family: "B Lotus";'>بسم الله الرحمن الرحیم </center> 
</h1> 

<h1 style='direction:rtl; font-family: "B Lotus";'> 
    <center style ='font-family: "B Lotus";'>طراحی سیستم بازیابی اطلاعات </center> 
</h1> 

<br>

<div style='direction:rtl; font-family: "B Lotus"; font-size: 18px'>
    <center> Contact: Amin Saeidi - amin.saeidi.1997@gmail.com - 400211579 </center>
    <center> Contact: Zahra Fazli  -z.fazli124@yahoo.com - 401301196 </center>
    <center> Contact: Hamid Reza Akbari - hakbari@gmail.com - 401301543</center>
</div>


<h1 style='direction:rtl; font-family: "B Lotus";'> 
    <center style ='font-family: "B Lotus";'>هدف و تعریف مسئله </center> 
</h1> 

<div style='direction:rtl; font-family: "B Lotus"; font-size: 18px'>
هدف این مساله پیدا کردن اسناد مرتبط با کویری و صحت سنجی نتایج به دست آمده می‌باشد.  این کار به سه روش مختلف پیاده سازی شده است. در روش اول با استفاده از tf-idf اسناد جاسازی شده‌اند و کوئری روی آن‌ها اعمال شده است. در روش دوم این جاسازی به کمک برت فارسی انجام شده است و در روش سوم، به جای میانگین گرفتن روی جاسازی تمام کلمات، به کمک tf-idf، میانگین وزن دار روی جاسازی‌های برت انجام گرفته تا به جاسازی سند برسیم. در ادامه مراحل کار و نتایج توضیح داده می‌شود.
</div>

<br>


<h1 style='direction:rtl; font-family: "B Lotus";'> 
    <center style ='font-family: "B Lotus";'>نصب موارد مورد نیاز و فراخوانی</center> 
</h1> 

In [5]:
!pip install transformers
!pip install hazm

In [16]:
import codecs
import math
from os import listdir
from os.path import isfile, join

import numpy as np
import tqdm
import torch
import random as rand
from hazm import *
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoModel, AutoTokenizer , PreTrainedTokenizer
from sklearn.preprocessing import normalize

<h1 style='direction:rtl; font-family: "B Lotus";'> 
    <center style ='font-family: "B Lotus";'>خواندن اسناد و کويری‌ها و پیش پردازش  </center> 
</h1> 
<div style='direction:rtl; font-family: "B Lotus"; font-size: 16px'>
  <ol>
      <li>تابع (pre_processing_query): کارهای پیش‌پردازش اعم از نرمال سازی متن و ریشه یابی روی کوئری را انجام‌ می‌دهد.</li>
      <li>تابع (tokenize_normalize_lemmatize):  برای تمام اسناد خوانده شده کارهای پیش‌پردازشی مشتمل بر نرمال سازی متن و ریشه یابی را انجام می‌دهد. با توجه به نوع مسئله ریشه یابی کلمات در یافتن عبارت های مشابه موثر خواهد بود</li>
      <li>تابع (load_documents): فایل‌های موجود در مسیر برنامه را به دست آورده و لیست آنها را برای خواندن به تابع load_document_list می‌دهد. فرمت اطلاعات مشتمل بر کل متن، لیست جملات و لیست کلمات متن در نظر گرفته شده است</li>
      <li>تابع (load_documents_list): لیست فایل‌های داده شده را به ترتیب می‌خواند و همه‌ي اطلاعات خوانده شده را برمی‌گرداند</li>
      <li>تابع (load_evaluation_file): با توجه به فرمت فایل ارزیابی ارائه شده، محتوای مورد نظر را به کمک تابع extract_evaluation_data اطلاعات لازم را استخراج می‌کند. </li>
      <li>تابع (extract_evaluation_data): اطلاعات موجود در فایل ارزیابی را استخراج می‌کند</li>
      <li>تابع (find_end_query_index):بر اساس یک کوئری انتهای آن را جهت بررسی کلمات فیمابین پیدا می‌کند. </li>
      <li>تابع (is_token_query): با توجه به فرمت اطلاعات ارزیابی برای تعیین شروع یک کوئری از این تابع استفاده می کند. </li>

  </ol> 
  </div>

In [11]:
class LoadAndPreprocessing:
    path_data = './IR_dataset'
    evaluation_data = 'evaluation_IR.yml'
    evaluation_reserve_word = ['relevant', 'similar_high', 'similar_low', 'similar_med']

    def __init__(self):
        self.normalizer = Normalizer()
        self.lemmatizer = Lemmatizer()

    def pre_processing_query(self, query):
        query = self.normalizer.normalize(query)
        words_list = [self.lemmatizer.lemmatize(w) for w in word_tokenize(query)]
        return ' '.join(words_list)

    def tokenize_normalize_lemmatize(self, data):
        for filename in tqdm.tqdm(data):
            x = data[filename]
            text = x['content']
            text = self.normalizer.normalize(text)

            sent_list = sent_tokenize(text)
            x['sentences'] = sent_list

            for sent in sent_list:
                x['words'] = [[self.lemmatizer.lemmatize(w)] for w in word_tokenize(sent)]

        return data

    def load_documents(self):
        training_docs = [f for f in listdir(self.path_data) if isfile(join(self.path_data, f))]
        training_data = self.load_documents_list(training_docs)
        return training_data

    def load_documents_list(self, filename_list):
        data = {}
        for file_name in tqdm.tqdm(filename_list):
            text = codecs.open(f"{self.path_data}/{file_name}", 'r', 'utf-8').read()
            data[file_name] = {'content': text}

        return data

    def load_evaluation_file(self):
        lines = codecs.open(self.evaluation_data, 'r', 'utf-8').readlines()
        token_list = []
        for line in lines:
            line = line.replace(':', '')
            line = line.replace('-', '')
            line = line.strip()
            token_list.append(line)

        data = {}
        start_query_index = 0
        while start_query_index < len(token_list):
            query = token_list[start_query_index]
            end_query_index = self.find_end_query_index(token_list, start_query_index)
            data_query = self.extract_evaluation_data(token_list[start_query_index + 1:end_query_index])
            query = self.pre_processing_query(query)
            data[query] = data_query
            start_query_index = end_query_index + 1

        return data

    def extract_evaluation_data(self, token_list):
        data = {}
        last_label = token_list[0]
        docs_list = []
        for index in range(1, len(token_list)):
            if token_list[index] in self.evaluation_reserve_word:
                data[last_label] = docs_list
                last_label = token_list[index]
                docs_list = []
                continue

            docs_list.append(token_list[index])

        data[last_label] = docs_list
        return data

    def find_end_query_index(self, token_list, start_query_index):
        end_query_index = start_query_index + 1
        while end_query_index < len(token_list):
            if self.is_token_query(token_list[end_query_index]):
                return end_query_index - 1
            end_query_index += 1

        return len(token_list) + 1

    def is_token_query(self, token):
        if token.isnumeric():
            return False
        if token in self.evaluation_reserve_word:
            return False
        if token in [':', '-']:
            return False

        return True


<h1 style='direction:rtl; font-family: "B Lotus";'> 
    <center style ='font-family: "B Lotus";'> قسمت اول - tf/idf</center> 
</h1> 
<div style='direction:rtl; font-family: "B Lotus"; font-size: 16px'>
  <ol>
      <li>تابع (set_tfidf_word): به ازای هر کلمه مقدار tfidf آن را به دست میاورد. (مورد استفاده در قسمت سوم)</li>
      <li>تابع (token_to_tfidf): از آنجایی که برت با آی دی توکن‌ها کار می‌کند، یک دیکشنری از آی دی به مقدار tfidf می‌سازیم.( مورد استفاده در قسمت سوم) </li>
      <li>تابع (predict): تابع اصلی جستجوی اسناد مبتنی بر همه کوئری ها در این تابع انجام و نتیجه جستجو در این بخش گزارش می شود. توجه به این نکته ضروری است که در صورتیکه یکی از مستندات یافت شده در مجموع داده های ارزیابی مدل وجود داشته باشد عملیات جستجو بر روی مستند مورد نظر با موفقیت گزارش می شود.. اما برای درک بهتر دقت عملیات این تفضیل به تفکیک میزان شباهت مستندات نیز گزارش شده است. عموما دقت تفضیل از دقت کلی کمتر می باشد</li>
      <li>تابع (predict_query):بر اساس ماتریس شباهت که با روش TF-IDF تولید شده اندیس مستندات مربوطه را از ماتریس استخراج و سپس نام هریک از فایل های بازیابی شده را برای مقایسه در بخش ارزیابی ارائه می نماید  </li>
      <li>تابع (create_corpus): داده‌ی ورودی (محتوای فایل‌ها ) را به قالب لیست متن و لیست نام مستندات تهیه و ارائه می نماید </li>
      <li>تابع (largest_indices): در صورتیکه در پارامتر ورودی مقدار n تعیین شود تعداد n اندیس آرایه با بیشترین مقدار را که به صورت نزولی مرتب شده است برمی گرداند.</li>

  </ol> 
  </div>

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

class TfIdfVectorization:
    path = 'data'

    def __init__(self, number):
        self.vectorizer = TfidfVectorizer()
        self.number = number

    def set_tfidf_word( self , training_data):
      data, doc_map = self.create_corpus(training_data)
      cv = CountVectorizer()
      data = cv.fit_transform(data)
      tfidf_transformer = TfidfTransformer()
      tfidf_matrix = tfidf_transformer.fit_transform(data)
      word2tfidf = dict(zip(cv.get_feature_names(), tfidf_transformer.idf_))
      return word2tfidf

    def token_to_tfidf(self , word2tfidf):
      model_name = "HooshvareLab/bert-base-parsbert-uncased"
      tmptokenizer = AutoTokenizer.from_pretrained(model_name)
      dic = {}
      for x in word2tfidf :
         dic[tmptokenizer.convert_tokens_to_ids(x)] = word2tfidf[x]
      return dic

    def predict(self, training_data, evaluation_data):
        corpus, doc_map = self.create_corpus(training_data)
        vec = self.vectorizer.fit_transform(corpus)

        acurracy_set = {}
        all_type_acurracy = 0
        for query in tqdm.tqdm(evaluation_data):
            y_hat = self.predict_query(vec, doc_map, query)
            y = {}
            for type in evaluation_data[query]:
                r_type = set()
                for r in evaluation_data[query][type]:
                    r_type.add(r)
                y[type] = r_type

            found = False
            for type in y:
                intersect = y_hat.intersection(y[type])
                if type not in acurracy_set:
                    acurracy_set[type] = 0
                if len(intersect) > 0:
                    acurracy_set[type] += 1
                    found = True

            if found:
                all_type_acurracy += 1

        print()
        for type in acurracy_set:
            accuracy = round(100 * acurracy_set[type] / len(evaluation_data), 2)
            print(f"Accuracy For Type {type} = {accuracy}")

        print(f"Accuracy For All Type = {round(100 * all_type_acurracy / len(evaluation_data), 2)}")

    def predict_query(self, vec, doc_map, query):
        similarity = self.vectorizer.transform([query]).dot(vec.T)
        r_doc_set = set()
        for doc in self.largest_indices(similarity.toarray()[0], self.number)[0]:
            r_doc_set.add(doc_map[doc].replace('.txt', ''))
        return r_doc_set

    def create_corpus(self, data):
        corpus = []
        doc_map = []
        for filename in data:
            x = data[filename]
            corpus.append(x['content'])
            doc_map.append(filename)
        return corpus, doc_map

    def largest_indices(self, array, number):
        flat = array.flatten()
        indices = np.argpartition(flat, -number)[-number:]
        indices = indices[np.argsort(-flat[indices])]
        return np.unravel_index(indices, array.shape)

<h1 style='direction:rtl; font-family: "B Lotus";'> 
    <center style ='font-family: "B Lotus";'> قسمت دوم - bert فارسی</center> 
</h1> 
<div style='direction:rtl; font-family: "B Lotus"; font-size: 16px'>
  <ol>
      <li>تابع (predict): اسناد را با bert جاسازی می‌کند و نتایج کویری را به دست آورده و دقت را با توجه به اطلاعات ارزیابی محاسبه می‌کند.</li>
      <li>تابع (predict_query):بر اساس ماتریس شباهت که با روش بازنمایی برت تولید شده اندیس مستندات مربوطه را از ماتریس استخراج و سپس نام هریک از فایل های بازیابی شده را برای مقایسه در بخش ارزیابی ارائه می نماید  </li>
      <li>تابع (extract_document_embeding_matrix):جاسازی تمام اسناد را در یک ماتریس ذخیره می‌کند </li>
      <li>تابع (extract_query_embeding_matrix):نتایج جاسازی کویری‌ها را در یک ماتریس ذخیره می‌کند </li>
      <li>تابع (get_document_embeding): به ازای هر سند جاسازی را استخراج می‌کند</li>
      <li>تابع (split_sentences): اگر طول جملات بیشتر از مقدار مورد قبول برت یعنی ۵۱۲ بود، جملات را به قسمت‌های کوچکتر می‌شکند.</li>
      <li>تابع (largest_indices): در صورتیکه در پارامتر ورودی مقدار n تعیین شود تعداد n اندیس آرایه با بیشترین مقدار را که به صورت نزولی مرتب شده است برمی گرداند.</li>

  </ol> 
  </div>
  <h1 style='direction:rtl; font-family: "B Lotus";'> 
    <center style ='font-family: "B Lotus";'> قسمت سوم ترکیبی</center> 
</h1> 
<div style='direction:rtl; font-family: "B Lotus"; font-size: 16px'>
همان قسمت دوم است که دیکشنری ساخته شده در tfidf را هم به عنوان ورودی می‌گیرد و با mode = 1 
  اجرا می‌شود.
  ضرایبی که از دیکشنری برای هر توکن پیدا می‌کند را در جاسازی برت ضرب می‌کند و به این ترتیب برای جاسازی سند از میانگین وزن‌دار جاسازی توکن‌ها استفاده می‌کند.
  </div>

In [5]:
from transformers.file_utils import tf_required

class ParsBertEmbeding:
    model_name = "HooshvareLab/bert-base-parsbert-uncased"

    def __init__(self, number):
        self.model = AutoModel.from_pretrained(self.model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.vectorizer = TfidfVectorizer()
        self.number = number

    def predict(self,  training_data, evaluation_data , mode , dic = None):
        doc_emb_matrix, doc_filename_list = self.extract_document_embeding_matrix(training_data , mode , dic)
        query_emb_matrix, query_list = self.extract_query_embeding_matrix(evaluation_data , mode , dic)
        similarity_matrix = query_emb_matrix.dot(doc_emb_matrix.T)
        acurracy_set = {}
        all_type_acurracy = 0
        for query_index in range(0, len(query_list)):
            query = query_list[query_index]
            y_hat = self.predict_query(similarity_matrix, doc_filename_list, query_index)
            y = {}
            for type in evaluation_data[query]:
                r_type = set()
                for r in evaluation_data[query][type]:
                    r_type.add(r)
                y[type] = r_type

            found = False
            for type in y:
                intersect = y_hat.intersection(y[type])
                if type not in acurracy_set:
                    acurracy_set[type] = 0
                if len(intersect) > 0:
                    acurracy_set[type] += 1
                    found = True

            if found:
                all_type_acurracy += 1

        print()
        for type in acurracy_set:
            accuracy = round(100 * acurracy_set[type] / len(evaluation_data), 2)
            print(f"Accuracy For Type {type} = {accuracy}")

        print(f"Accuracy For All Type = {round(100 * all_type_acurracy / len(evaluation_data), 2)}")
        return similarity_matrix

    def predict_query(self, similarity_matrix, doc_filename_list, query_index):
        r_doc_set = set()
        for doc_index in self.largest_indices(similarity_matrix[query_index, :], self.number)[0]:
            r_doc_set.add(doc_filename_list[doc_index].replace('.txt', ''))
        return r_doc_set

    def extract_document_embeding_matrix(self, data , mode , dic):
        doc_emb_matrix = np.zeros((1, 768), dtype=float)
        doc_filename_list = list()
        for filename in tqdm.tqdm(data):
            doc = data[filename]
            doc_emb = self.get_document_embeding(doc , mode , dic)
            doc_emb_matrix = np.concatenate((doc_emb_matrix, doc_emb), axis=0)
            doc_filename_list.append(filename)

        doc_emb_matrix = doc_emb_matrix[1:, :]
        doc_emb_matrix_normed = normalize(doc_emb_matrix, axis=1, norm='l1')
        return doc_emb_matrix_normed, doc_filename_list

    def extract_query_embeding_matrix(self, evaluation_data , mode , dic):
        query_emb_matrix = np.zeros((1, 768), dtype=float)
        query_list = list()
        for query in tqdm.tqdm(evaluation_data):
            query_emb = torch.zeros((768), dtype=torch.float64)
            query_tokens = self.tokenizer(query, return_tensors="pt")
            with torch.no_grad():
                outputs = self.model(**query_tokens)
                if mode == 1 :
                  for x in range(len(query_tokens['input_ids'][0])):
                    if query_tokens['input_ids'][0][x].item() in dic:
                        outputs.last_hidden_state[0, :, :][x] = outputs.last_hidden_state[0, :, :][x]*dic[query_tokens['input_ids'][0][x].item()]
                    else :
                        outputs.last_hidden_state[0, :, :][x] = outputs.last_hidden_state[0, :, :][x]*0
                query_emb += torch.mean(outputs.last_hidden_state[0, :, :], dim=0)

            query_emb_matrix = np.concatenate((query_emb_matrix, query_emb[np.newaxis, :].numpy()), axis=0)
            query_list.append(query)

        query_emb_matrix = query_emb_matrix[1:, :]
        query_emb_matrix_normed = normalize(query_emb_matrix, axis=1, norm='l1')
        return query_emb_matrix_normed, query_list

    def get_document_embeding(self, doc , mode , dic ):
        sent_list = []
        for sent in doc['sentences']:
           sent_list.extend(self.split_sentences(sent))

        emb_matrix = torch.zeros((1, 768), dtype=torch.float64)
        for sent in sent_list:
            sent_tokens = self.tokenizer(sent, return_tensors="pt")
            with torch.no_grad():
                try:
                    outputs = self.model(**sent_tokens)
                    if mode == 1 :
                      for x in range(len(sent_tokens['input_ids'][0])):
                       if sent_tokens['input_ids'][0][x].item() in dic:
                        outputs.last_hidden_state[0, :, :][x] = outputs.last_hidden_state[0, :, :][x]*dic[sent_tokens['input_ids'][0][x].item()]
                       else :
                         outputs.last_hidden_state[0, :, :][x] = outputs.last_hidden_state[0, :, :][x]*0
                    emb_matrix = torch.cat((emb_matrix, outputs.last_hidden_state[0, :, :]), axis=0)
                except:
                    print("An exception occurred")

        emb_matrix_avg = torch.mean(emb_matrix, dim=0)
        return emb_matrix_avg[np.newaxis, :].numpy()

    def split_sentences(self, sent):
        word_in_sentence = 450
        sent_list = []
        word_token_list = word_tokenize(sent)
        if len(word_token_list) < word_in_sentence:
            sent_list.append(sent)
            return sent_list

        num_sent = math.ceil(len(word_token_list) / word_in_sentence)
        start = 0
        end = start + word_in_sentence
        for index in range(0, num_sent):
            sent_list.append(' '.join(word_token_list[start:end]))
            start = end + 1
            end = min(start + word_in_sentence, len(word_token_list))
        return sent_list

    def largest_indices(self, array, number):
        flat = array.flatten()
        indices = np.argpartition(flat, -number)[-number:]
        indices = indices[np.argsort(-flat[indices])]
        return np.unravel_index(indices, array.shape)


<div style='direction:rtl; font-family: "B Lotus"; font-size: 18px'>
فایل‌ها را از مسیر مورد نظر خوانده و اطلاعات را در training_data نگه داری می‌کند. هم چنین اطلاعات فایل ارزیابی را در evaluation_data نگه می‌دارد.
</div>

In [13]:
loader = LoadAndPreprocessing()
training_data = loader.load_documents()
training_data = loader.tokenize_normalize_lemmatize(training_data)
evaluation_data = loader.load_evaluation_file()

100%|█████████████████████████████████████████████████████████████████████████████| 3258/3258 [00:06<00:00, 511.45it/s]


<h1 style='direction:rtl; font-family: "B Lotus";'> 
    <center style ='font-family: "B Lotus";'>نتایج قسمت اول </center> 
</h1> 
<div style='direction:rtl; font-family: "B Lotus"; font-size: 18px'>
مربوط به اجرای tfidf است. همان طور که از نتایج مشخص است این روش می‌تواند با دقت قابل قبولی روی اسناد داده شده جستجو را انجام دهد
</div>

In [7]:
from numpy.lib.function_base import vectorize

vectorizer = TfIdfVectorization(20)
word_to_value_dic = vectorizer.set_tfidf_word(training_data)
token_to_tfidf = vectorizer.token_to_tfidf(word_to_value_dic)
vectorizer.predict(training_data, evaluation_data)

c:\Users\zahra\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
100%|██████████| 150/150 [00:04<00:00, 31.57it/s]


Accuracy For Type relevant = 91.33
Accuracy For Type similar_high = 90.67
Accuracy For Type similar_low = 0.0
Accuracy For Type similar_med = 84.0
Accuracy For All Type = 96.67


<h1 style='direction:rtl; font-family: "B Lotus";'> 
    <center style ='font-family: "B Lotus";'>نتایج قسمت دوم و سوم </center> 
</h1> 
<div style='direction:rtl; font-family: "B Lotus"; font-size: 18px'>
نتایج اول مربوط به اجرای الگوریتم با مود ۰ است که در واقع فقط از برت برای جاسازی اسناد استفاده می‌کند. همان طور که از نتایج مشخص است این قسمت نسبت
به tfidf از دقت کمتری برخوردار است ۸۴ درصد به نسبت ۹۷ درصد در tfidf. زمان اجرا هم در این حالت به صورت چشمگیری نسبت به tfidf  بالاست.
در این قسمت هر سند به صورت جمله به جمله به برت داده می‌شود و برای به دست آوردن جاسازی سند میانگین جاسازی تمام توکن‌ها به عنوان جاسازی سند در نظر گرفته می‌شود. 
وقتی به جای میانگین با کمک tfidf میانگین وزن‌دار می‌گیریم، دقت کار افزایش پیدا می‌کند (ار ۸۴ درصد به ۹۲ درصد افزایش می‌یابد)و علت آن هم این است که تمام کلمات موجود در متن ارزش یکسانی ندارند و با دادن ضریب به کلمات مهم‌تر، نتیجه‌ی دقیق‌تری تولید می‌شود.
</div>

In [10]:
bert = ParsBertEmbeding(20)
bert.predict(training_data, evaluation_data , 0)
bert.predict(training_data, evaluation_data  , 1 , token_to_tfidf )


Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
 17%|█▋        | 548/3258 [43:38<5:40:36,  7.54s/it]

An exception occurred


 17%|█▋        | 549/3258 [43:47<5:59:21,  7.96s/it]

An exception occurred


 17%|█▋        | 570/3258 [45:13<2:38:41,  3.54s/it]

...

 99%|█████████▉| 3236/3258 [4:41:52<01:54,  5.19s/it]  

An exception occurred
An exception occurred
An exception occurred


100%|██████████| 150/150 [00:19<00:00,  7.73it/s]



Accuracy For Type relevant = 58.67
Accuracy For Type similar_high = 77.33
Accuracy For Type similar_low = 3.33
Accuracy For Type similar_med = 72.67
Accuracy For All Type = 92.0


array([[0.00172171, 0.00158113, 0.00152524, ..., 0.0014316 , 0.00141329,
        0.00159319],
       [0.00135551, 0.00127134, 0.0012751 , ..., 0.00132523, 0.00128545,
        0.00150147],
       [0.00160676, 0.00146611, 0.00140225, ..., 0.0018339 , 0.00180934,
        0.00182751],
       ...,
       [0.00133119, 0.00126054, 0.00123567, ..., 0.00134421, 0.00131315,
        0.0014227 ],
       [0.00147329, 0.00135558, 0.00139093, ..., 0.00152353, 0.00150857,
        0.00148624],
       [0.0013731 , 0.00125534, 0.00124769, ..., 0.00135081, 0.00133622,
        0.00135603]])

<h1 style='direction:rtl; font-family: "B Lotus";'> 
    <center style ='font-family: "B Lotus";'>قسمت امتیازی - طبقه بندی</center> 
</h1> 
<div style='direction:rtl; font-family: "B Lotus"; font-size: 18px'>
در این بخش هدف این است تا در ابتدا به وسیله امبدینگ خروجی برت فارسی، ابتدا مجموعه داده ای از زوج های (پرسمان مرتبط - سند) و (پرسمان نامرتبط - سند) ایجاد کنیم و سپس با استفاده از این مجموعه داده یک دسته بند ایجاد کنیم و بتوانیم دادگان را از منظر گفته شده طبقه بندی کنیم. برای بخش ساخت مجموعه داده به کلاس پارس-برت-امبدینگ دو تابع اضافه شده است که در ادامه توضیحات آن ها آورده شده است. 
</div>

<div style='direction:rtl; font-family: "B Lotus"; font-size: 16px'>
  <ol>
      <li>تابع (extract_both_embeding_matrices): این تابع ماتریس امدبینگ سند و کوئری ها را  مطابق با توضیحات قسمت های قبل استخراج کرده و برای استفاده در تابع بعدی به عنوان خروجی میدهد. </li>
      <li>تابع (making_dataset): این تابع داده ست های ذکر شده را استحراج میکند. روال کار به این صورت است که در کل با توجه به فایل ارزیابی داده شده، دو بردار خروجی تولید میکند، یکی بردار ایکس و دیگری بردار ایگرگ. هر سطر از بردار ایکس کانکت شده امبدینگ یک سند و یک کوئری است که این امبدینگ ها خروجی پارس برت هستند. به همین ترتیب در هر سطر متناظر بردار ایگرگ یک عدد برای مشخص کردن این که سند و کوئری هر سطر ایکس با یکدیگر مرتبط هستند یا خیر قرار میگیرد. لازم به ذکر است که برای هر کوئری 70 سند در ایکس قرار گرفته اند که از این 70 تا سند، تعدادی سند های مرتبط اند که از فایل ارزیابی استخراج شده اند و باقی مانده ان ها به صورت تصادفی از بقیه سند های نامرتبط انتخاب شده اند.     </li>
  </ol> 
  </div>

</h1> 
<div style='direction:rtl; font-family: "B Lotus"; font-size: 18px'>
در ادامه باری دیگر کلاس پارس پرت امبدینگ صرفا با تابع های مورد نیاز این بخش اورده شده است.البته میتوان این تابع ها را به کلاس اصلی اضافه کرده و نیاز به بازنویسی مجدد نباشد، اما برای شفاف تر بودن طریقه تولید دیتاست بار دیگر کلاس آورده شده است. 
</div>

In [9]:
class ParsBertEmbeding_c:
    model_name = "HooshvareLab/bert-base-parsbert-uncased"

    def __init__(self):
        self.model = AutoModel.from_pretrained(self.model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
    
    def extract_both_embeding_matrices(self, training_data, evaluation_data):
        doc_emb_matrix, doc_filename_list = self.extract_document_embeding_matrix(training_data)
        query_emb_matrix, query_list = self.extract_query_embeding_matrix(evaluation_data)
        
        return doc_emb_matrix, doc_filename_list, query_emb_matrix, query_list
    
    def making_dataset(self, doc_emb_matrix, query_emb_matrix, doc_filename_list, query_list, evaluation_data):
        
        doc_filename_list = [int(w.replace(".txt","")) for w in doc_filename_list]
        
        number_of_docs = len(doc_filename_list)
        number_of_queries = len(query_list)
        
        each_query_no = 70
        embeding_length = len(doc_emb_matrix[0])
        columns = 2 * embeding_length
        rows = each_query_no * number_of_queries
        
        x_vector = np.zeros((rows,columns))
        y_vector = np.zeros((rows,1))
            
        # for each query, we get the relevant, similar_high, similar_med-> relevant and assign 1 for their y 
        # for each query, we produce 70 data for our training data, some from above, some random from rest
        over_all_counter = 0
        for query in query_list:
            counter = 0
            pre_doc = []
            for doc in evaluation_data[query]['relevant']:
                doc_name = int(doc)
                doc_index = doc_filename_list.index(doc_name)
                pre_doc.append(doc_index)
                query_index = query_list.index(query)
                mixed_embedding = np.concatenate((doc_emb_matrix[doc_index],query_emb_matrix[query_index]))
                x_vector[over_all_counter] = mixed_embedding
                y_vector[over_all_counter] = 1
                counter += 1
                over_all_counter += 1

            for doc in evaluation_data[query]['similar_high']:
                doc_name = int(doc)
                doc_index = doc_filename_list.index(doc_name)
                pre_doc.append(doc_index)
                query_index = query_list.index(query)
                mixed_embedding = np.concatenate((doc_emb_matrix[doc_index],query_emb_matrix[query_index]))
                x_vector[over_all_counter] = mixed_embedding
                y_vector[over_all_counter] = 1
                counter += 1
                over_all_counter += 1

            for doc in evaluation_data[query]['similar_med']:
                doc_name = int(doc)
                doc_index = doc_filename_list.index(doc_name)
                pre_doc.append(doc_index)
                query_index = query_list.index(query)
                mixed_embedding = np.concatenate((doc_emb_matrix[doc_index],query_emb_matrix[query_index]))
                x_vector[over_all_counter] = mixed_embedding
                y_vector[over_all_counter] = 1
                counter += 1
                over_all_counter += 1
            
            while counter < each_query_no:
                random_index = rand.randint(0, number_of_docs-1)
                if random_index not in pre_doc:
                    doc_index = random_index
                    pre_doc.append(doc_index)
                    query_index = query_list.index(query)
                    mixed_embedding = np.concatenate((doc_emb_matrix[doc_index],query_emb_matrix[query_index]))
                    x_vector[over_all_counter] = mixed_embedding
                    y_vector[over_all_counter] = 0
                    counter += 1
                    over_all_counter += 1
    
        return x_vector, y_vector
            
            
    def extract_document_embeding_matrix(self, data):
        doc_emb_matrix = np.zeros((1, 768), dtype=float)
        doc_filename_list = list()
        for filename in tqdm.tqdm(data):
            doc = data[filename]
            doc_emb = self.get_document_embeding(doc)
            doc_emb_matrix = np.concatenate((doc_emb_matrix, doc_emb), axis=0)
            doc_filename_list.append(filename)

        doc_emb_matrix = doc_emb_matrix[1:, :]
        doc_emb_matrix_normed = normalize(doc_emb_matrix, axis=1, norm='l1')
        return doc_emb_matrix_normed, doc_filename_list

    def extract_query_embeding_matrix(self, evaluation_data):
        query_emb_matrix = np.zeros((1, 768), dtype=float)
        query_list = list()
        for query in tqdm.tqdm(evaluation_data):
            query_emb = torch.zeros((768), dtype=torch.float64)
            query_tokens = self.tokenizer(query, return_tensors="pt")
            with torch.no_grad():
                outputs = self.model(**query_tokens)
                query_emb += torch.mean(outputs.last_hidden_state[0, :, :], dim=0)

            query_emb_matrix = np.concatenate((query_emb_matrix, query_emb[np.newaxis, :].numpy()), axis=0)
            query_list.append(query)

        query_emb_matrix = query_emb_matrix[1:, :]
        query_emb_matrix_normed = normalize(query_emb_matrix, axis=1, norm='l1')
        return query_emb_matrix_normed, query_list

    def get_document_embeding(self, doc):
        sent_list = []
        for sent in doc['sentences']:
            sent_list.extend(self.split_sentences(sent))

        emb_matrix = torch.zeros((1, 768), dtype=torch.float64)
        for sent in sent_list:
            sent_tokens = self.tokenizer(sent, return_tensors="pt")
            with torch.no_grad():
                try:
                    outputs = self.model(**sent_tokens)
                    emb_matrix = torch.cat((emb_matrix, outputs.last_hidden_state[0, :, :]), axis=0)
                except:
                    print("An exception occurred")

        emb_matrix_avg = torch.mean(emb_matrix, dim=0)
        return emb_matrix_avg[np.newaxis, :].numpy()

    def split_sentences(self, sent):
        word_in_sentence = 450
        sent_list = []
        word_token_list = word_tokenize(sent)
        if len(word_token_list) < word_in_sentence:
            sent_list.append(sent)
            return sent_list

        num_sent = math.ceil(len(word_token_list) / word_in_sentence)
        start = 0
        end = start + word_in_sentence
        for index in range(0, num_sent):
            sent_list.append(' '.join(word_token_list[start:end]))
            start = end + 1
            end = min(start + word_in_sentence, len(word_token_list))
        return sent_list

In [14]:
bert_c = ParsBertEmbeding_c()
doc_emb_matrix, doc_filename_list, query_emb_matrix, query_list = bert_c.extract_both_embeding_matrices(training_data, evaluation_data)

Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
 17%|████████████▉                                                                | 548/3258 [09:46<1

An exception occurred



 17%|████████████▉                                                                | 549/3258 [09:48<1:24:07,  1.86s/it]

An exception occurred


 17%|█████████████▊                                                                 | 570/3258 [10:07<33:58,  1.32it/s]

...

 85%|██████████████████████████████████████████████████████████████████▍           | 2774/3258 [48:13<05:24,  1.49it/s]

An exception occurred


 99%|█████████████████████████████████████████████████████████████████████████████▍| 3236/3258 [55:30<00:23,  1.05s/it]

An exception occurred
An exception occurred
An exception occurred


100%|████████████████████████████████████████████████████████████████████████████████| 150/150 [00:06<00:00, 22.79it/s]


In [17]:
x_dataset, y_dataset = bert_c.making_dataset(doc_emb_matrix, query_emb_matrix, doc_filename_list, query_list, evaluation_data)

<h1 style='direction:rtl; font-family: "B Lotus";'> 
    <center style ='font-family: "B Lotus";'>ادامه قسمت امتیازی و انجام عملیات طبقه بندی</center> 
</h1> 
<div style='direction:rtl; font-family: "B Lotus"; font-size: 18px'>
در این قسمت با استفاده از دیتاست های استخراج شده در مرحله قبل و با استفاده از لاجیستیک رگرسیون یک طبقه بندی بر روی داده ها صورت گرفته است که در نهایت به امتیاز 79 درصد داده ها را برای ما طبقه بندی میکند. لازم به ذکر است که از دیتاست تولید شده در مرحله قبل 80 درصد برای آموزش لاجیستیک رگرسیون و 20 درصد برای تست مدل آموزش دیده استفاده شده است.
کد و خروجی مرتبط با این قسمت در ادامه آورده شده است. 
</div>

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_dataset, y_dataset, random_state=0, train_size = 0.8)

In [19]:
LR_classifier = LogisticRegression()
LR_classifier.fit(x_train, np.ravel(y_train))
print("Classification score is :" + str(LR_classifier.score(x_test,y_test)))

Classification score is :0.7880952380952381
